In [ ]:
from browser import Browser
import time
import pandas as pd
import numpy as np
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
os.makedirs("Data/", exist_ok=True)
os.makedirs("Data/Listed", exist_ok=True)

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
a = Browser(False)

In [ ]:
def go_to_id(a: Browser, target_id):
    a.driver.get("https://www2.jpx.co.jp/tseHpFront/JJK020010Action.do")
    time.sleep(2)

    target_page = target_id // 100 + 1

    select = Select(a.driver.find_element(By.NAME, "dspSsuPd"))
    select.select_by_value("100")
    for checkbox in a.driver.find_elements(By.NAME, "szkbuChkbx"):
        checkbox.click()

    time.sleep(1)
    a.driver.find_element(By.CLASS_NAME, "activeButton").click()
    time.sleep(2)

    if target_page != 1:
        current_page = 1
        while True:
            paging_menu = a.driver.find_element(By.CLASS_NAME, "pagingmenu")
            list_button = paging_menu.find_elements(By.TAG_NAME, "a")
            list_num = []
            for button in list_button:
                try:
                    num = int(button.text)
                except:
                    num = -1

                list_num.append(num)

            try:
                btn_idx = list_num.index(target_page)
            except:
                btn_idx = -1

            if btn_idx  == -1:
                current_page = max(list_num) + 1
                print(current_page)
                for button in list_button[::-1]:
                    if button.text == "...":
                        button.click()
                        time.sleep(2)
                        break
                if current_page == target_page:
                    break
            else:
                current_page = target_page
                print(current_page)
                list_button[btn_idx].click()
                time.sleep(2)
                break

    a.scroll_to_bottom()

In [ ]:
def get_data(a: Browser, target_id):
    soup = a.get_soup()
    table = soup.find("table", {"class":"tableStyle01"})
    data = pd.read_html(str(table))[0]
    data["Date of incorporation"] = None
    temp = []
    for c_ in data["Code"]:
        if type(c_) == float or type(c_) == int:
            temp.append(str(int(c_)))
        else:
            temp.append(str(c_))
    
    data["Code"] = temp
    list_button_detail = a.driver.find_elements(By.NAME, "detail_button")
    N = len(list_button_detail)
    for i in range(N):
        list_button_detail = a.driver.find_elements(By.NAME, "detail_button")
        list_button_detail[i].click()
        time.sleep(1)

        soup = a.get_soup()
        list_table = soup.find_all("table")
        DF = None
        for i in range(len(list_table)):
            try:
                df = pd.read_html(str(list_table[i]))[0]
            except:
                continue
            try:
                DF = pd.concat([DF, df], axis=1)
            except:
                DF = df.copy()
        
        try:
            code = DF.loc[0, "Code"]
            if type(code) == pd.Series:
                code = code[0]

            if type(code) == float or type(code) == int:
                code = str(int(code))
            else:
                code = str(code)
            
            if code.endswith(".0"): code = code[:-2]

            date_of_incorporation = DF.loc[0, "Date of incorporation"]
            if type(date_of_incorporation) == pd.Series:
                date_of_incorporation = date_of_incorporation[0]
        except:
            code, date_of_incorporation = None, None
        
        print(code, date_of_incorporation)
        if code is not None: data.loc[data["Code"]==code, "Date of incorporation"] = date_of_incorporation
        data.to_csv(f"Data/Listed/{target_id}.csv", index=False)
        a.driver.back()
        time.sleep(0.5)
        a.scroll_to_bottom(wait_page_load=0.5)


In [ ]:
from IPython.display import clear_output
clear_output()
a.reset_browser()

for target_id in range(0, 4355, 100):
    go_to_id(a, target_id)
    get_data(a, target_id)
    a.reset_browser()
    clear_output()